In [1]:
import numpy as np
from rascal.representations import SphericalExpansion as SPH
import ase.io
import tqdm
from nice.blocks import *
from nice.utilities import *
from matplotlib import pyplot as plt
from sklearn.linear_model import BayesianRidge
import torch
torch.set_num_threads(1)
import torch.nn
import sys
sys.path.append('./code/')
from code_pytorch import *
import time
import numpy as np
from sympy.physics.wigner import wigner_3j
from sympy.physics.quantum.spin import Rotation as rot
from rascal.representations import SphericalInvariants as SOAP
from torch2trt import torch2trt

In [2]:
class Powerspectrum(torch.nn.Module):
    def __init__(self, clebsch):
        super(Powerspectrum, self).__init__()
        self.first = ClebschCombining(clebsch, 0)       
            
    def forward(self, X):
        ps_invariants = self.first(X, X)       
        return ps_invariants
    
class Bispectrum(torch.nn.Module):
    def __init__(self, clebsch, lambda_max):
        super(Bispectrum, self).__init__()
        self.first = ClebschCombining(clebsch, lambda_max)
        self.second = ClebschCombining(clebsch, 0)
            
    def forward(self, X):
        ps_covariants = self.first(X, X)
        bs_invariants = self.second(ps_covariants, X)
        return bs_invariants
    
def convert_to_torch(coefficients, device):
    result = {}
    for lambd in range(coefficients.shape[2]):
        result[lambd] = torch.FloatTensor(coefficients[:, :, lambd, : 2 * lambd + 1]).to(device)
        result[lambd] = result[lambd].transpose(0, -1)
    return result

In [3]:
LAMBDA_MAX = 5

HYPERS = {
    'interaction_cutoff': 6.3,
    'max_radial': 5,
    'max_angular': LAMBDA_MAX,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.3,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'
}

HYPERS_PS = copy.deepcopy(HYPERS)
HYPERS_PS['soap_type'] = 'PowerSpectrum'

HYPERS_BS = copy.deepcopy(HYPERS)
HYPERS_BS['soap_type'] = 'BiSpectrum'
HYPERS_BS['inversion_symmetry'] = False

In [4]:
def get_rascal_ps(structures):
    #nice.rascal_coefficients.process_structures(structures)
    soap_ps = SOAP(**HYPERS_PS)   
    ps = soap_ps.transform(structures).get_features(soap_ps)
    return ps

def get_rascal_bs(structures):
    soap_bs = SOAP(**HYPERS_BS)    
    bs = soap_bs.transform(structures).get_features(soap_bs)
    return bs

def get_torch_invariants(structures, device, model):
    all_species = get_all_species(structures)
    coefficients = get_spherical_expansion(structures, HYPERS,
                                             all_species, show_progress=False)    
    
    result = {}
    for specie in all_species:
        result[specie] = model(convert_to_torch(coefficients[specie], device))
        
    return result

# time PS computation

In [5]:
structures = ase.io.read('methane.extxyz', index='0:1000')
structures = nice.rascal_coefficients.process_structures(structures) #put all inside unit cell for rascal
clebsch = nice.clebsch_gordan.ClebschGordan(LAMBDA_MAX)

In [6]:
%%timeit
_ = get_rascal_ps(structures)

157 ms ± 730 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
model_ps = Powerspectrum(clebsch.precomputed_)

In [8]:
%%timeit
_ = get_torch_invariants(structures, 'cpu', model_ps)

521 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
model_ps = model_ps.cuda()

In [10]:
%%timeit
_ = get_torch_invariants(structures, 'cuda', model_ps)

552 ms ± 9.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# time BS computation

In [11]:
structures = ase.io.read('methane.extxyz', index='0:100')
structures = nice.rascal_coefficients.process_structures(structures) #put all inside unit cell for rascal
clebsch = nice.clebsch_gordan.ClebschGordan(LAMBDA_MAX)

In [12]:
%%timeit
_ = get_rascal_bs(structures)

33.4 s ± 34.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
model_bs = Bispectrum(clebsch.precomputed_, LAMBDA_MAX)

In [14]:
%%timeit
_ = get_torch_invariants(structures, 'cpu', model_bs)

8 s ± 187 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
model_bs = model_bs.cuda()

In [16]:
%%timeit
_ = get_torch_invariants(structures, 'cuda', model_bs)

1.57 s ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
